<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/answer_Generation_ALLaM_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [2]:
import os
import torch
import json
import pandas as pd
from google.colab import files
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from IPython.display import FileLink, display
from datasets import load_dataset

In [5]:
torch.cuda.empty_cache()

In [3]:
def setup_allam_model():
    """Set up the ALLaM model and tokenizer"""
    model_id = "ALLaM-AI/ALLaM-7B-Instruct-preview"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id)

    # Move model to GPU if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)

    return model, tokenizer, device

def generate_chat_response(user_message, model, tokenizer, device):
    """Generate a response using the chat template approach"""
    messages = [
        {"role": "user", "content": user_message},
    ]

    inputs = tokenizer.apply_chat_template(messages, tokenize=False)
    inputs = tokenizer(inputs, return_tensors='pt', return_token_type_ids=False)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    response = model.generate(
        **inputs,
        max_new_tokens=4096,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.6
    )

    full_response = tokenizer.batch_decode(response, skip_special_tokens=True)[0]

    # Extract just the model's response (removing the input)
    model_response = full_response[len(inputs):].strip()

    return model_response

In [4]:
def generate_answer(question, num_samples, model, tokenizer):
    # Create the prompt with the question
    prompt = f"أجب على السؤال التالي بجملة واحدة فقط موجزة ولكن كاملة باللغة العربية\nQuestion: {question}\nAnswer:"

    # Tokenize the prompt without adding special tokens
    prompt_tokens = tokenizer(prompt, add_special_tokens=False, return_tensors="pt").to(model.device)
    prompt_length = prompt_tokens.input_ids.shape[1]  # Get actual prompt length

    # Generate responses using the model
    outputs = model.generate(
        **prompt_tokens,
        max_length=150,
        do_sample=True,
        temperature=0.5,
        top_p=0.95,
        num_return_sequences=num_samples,
        return_dict_in_generate=True,
        output_scores=True,
    )

    sequences = outputs.sequences
    results = []

    for seq in sequences:
        # Decode and clean the generated text
        decoded_text = tokenizer.decode(seq, skip_special_tokens=True)

        # Extract the answer after the "Answer:" marker
        answer = decoded_text.split("Answer:")[-1].strip()

        # Clean the output
        strings_to_filter_on = ['.', '\n', 'Q:', 'A:', 'question:', 'answer:', 'Question:', 'Answer:', 'Questions:', 'questions:', 'QUESTION:', 'ANSWER:']
        for string in strings_to_filter_on:
            if string in answer:
                answer = answer.split(string)[0].strip()

        # Calculate negative log-likelihood
        with torch.no_grad():
            # Create target labels with prompt masked
            target_ids = seq.clone()
            target_ids[:prompt_length] = -100  # Ignore prompt tokens in loss

            # Forward pass to compute loss
            output = model(seq.unsqueeze(0), labels=target_ids.unsqueeze(0))

            # Calculate total NLL for generated tokens
            avg_nll_per_token = output.loss
            num_generated_tokens = len(seq) - prompt_length
            total_nll = avg_nll_per_token * num_generated_tokens

        results.append({
            'text': answer,
            'neg_log_likelihood': total_nll.item(),
        })

    # Sort results by likelihood (lower NLL is better)
    results.sort(key=lambda x: x['neg_log_likelihood'])
    return results

In [5]:
def load_qa_dataset(dataset_name, file_path=None):
    data = []

    try:
        if dataset_name == 'arabicaqa' and file_path:
            if os.path.exists(file_path):  # Use the loaded file
                print("Using ArabicaQA")
                with open(file_path, 'r', encoding='utf-8') as f:
                    custom_data = json.load(f)
                    for idx, item in enumerate(custom_data):
                        data.append({
                            "question_id": idx,
                            "Question": item["question"],
                            "Answer": item["answer"]
                        })
            else:
                raise FileNotFoundError(f"ArabicaQA file not found at {file_path}")

        elif dataset_name == 'xor_tydiqa' and file_path:
            if os.path.exists(file_path):  # Use the loaded file
                print("Using XOR-TyDiQA")
                print("Filtering Arabic QA pairs from XOR-TyDi...")

                # Load the jsonl dataset
                with open(file_path, 'r', encoding='utf-8') as f:
                    custom_data = [json.loads(line) for line in f]  # Handling jsonl

                # Filter Arabic samples ("lang" = "ar")
                arabic_data = []
                for idx, item in enumerate(custom_data):
                    if item["lang"] == "ar":  # Arabic language code
                        arabic_data.append({
                            "question_id": item["id"],
                            "Question": item["question"],
                            "Answer": item["answers"][0]  # First answer in list
                        })

                data.extend(arabic_data)
            else:
                raise FileNotFoundError(f"XOR-TyDiQA file not found at {file_path}")

        elif dataset_name == 'xquad':
            print("Using XQuAD")
            dataset = load_dataset("xquad", "xquad.ar")
            for idx, item in enumerate(dataset["validation"]):  # Only available set
                data.append({
                    "question_id": f"xquad_{idx}",
                    "Question": item["question"],
                    "Answer": item["answers"]["text"][0]
                })

        elif dataset_name == 'arcd':
            print("Using ARCD")
            dataset = load_dataset("arcd")
            for split in ["train", "validation"]:
                for idx, item in enumerate(dataset[split]):
                    data.append({
                        "question_id": f"arcd_{idx}",
                        "Question": item["question"],
                        "Answer": item["answers"]
                    })

        elif dataset_name == 'arabic_squad':
            print("Using Arabic SQuAD")
            dataset = load_dataset("Mostafa3zazi/Arabic_SQuAD")
            for split in ["train"]:
                for idx, item in enumerate(dataset[split]):
                    data.append({
                        "question_id": f"squad_{idx}",
                        "Question": item["question"],
                        "Answer": item["text"]
                    })

        elif dataset_name == 'mintaka':
            print(f"Using Mintaka in arabic")
            dataset = load_dataset("AmazonScience/mintaka", "ar")
            if "train" in dataset:
                dataset = dataset["train"]  # Select training split
            for idx, item in enumerate(dataset):
                    data.append({
                        "question_id": item['id'],
                        "Question": item['question'],
                        "Answer": item['answerText']
                    })

        elif dataset_name == 'aqad' and file_path:
            if os.path.exists(file_path):
                print("Using AQAD")
                # Load the dataset
                with open(file_path, 'r', encoding='utf-8') as f:
                    dataset = json.load(f)  # Assuming the dataset is in JSON format

                # Loop through the data structure
                for article in dataset.get('data', []):  # Ensure to check 'data'
                    title = article.get('title', 'Unknown Title')  # Handle missing title
                    for paragraph in article.get('paragraphs', []):  # Ensure to check 'paragraphs'
                        for qas in paragraph.get('qas', []):  # Ensure to check 'qas'
                            question = qas.get('question', '').strip()  # Handle missing question
                            answers = qas.get('answers', [])
                            if answers:
                                answer_text = answers[0].get('text', '').strip()  # Handle missing answer
                                if question and answer_text:  # Ensure both question and answer are present
                                    data.append({
                                        "question_id": f"aqad_{qas.get('id', 'unknown')}",  # Handle missing ID
                                        "Question": question,
                                        "Answer": answer_text
                                    })
                            else:
                                print(f"Skipping entry with missing answer for question: {question}")

        else:
            raise ValueError(f"Unsupported dataset: {dataset_name}")

    except Exception as e:
        print(f"Error loading {dataset_name}: {str(e)}")
        return []

    return data

In [7]:
 # Main function
def main():
    # Step 1: SELECT AND LOAD DATA
    DATASET_CHOICE = 'xor_tydiqa'  # Choose from above

    FILE_PATH = '/content/xor_dev_full_v1_1.jsonl'  # Only needed for ArabicaQA And xor_tydiqa and AAQAD

    data = load_qa_dataset(DATASET_CHOICE, FILE_PATH if DATASET_CHOICE == 'arabicaqa' or 'xor_tydiqa' else None)

    if not data:
        print("No data loaded. Exiting...")
        return

    # Convert to DataFrame
    df = pd.DataFrame(data)[["question_id", "Question", "Answer"]]
    print(f"Loaded {len(df)} questions\n")

  # Step 2: GENERATE ANSWERS
  #empty list to store the results
    results = []

    for index, row in df.iterrows():
        question = row['Question']
        original_answer = row.get('Answer',"")  # Use empty string if no original answer provided
        question_id = row.get('question_id')

        # Generate 10 answers with log probabilities for each question
        generated_results = generate_answer(
            question=question,
            num_samples=10,
            model= setup_allam_model(),
            tokenizer= setup_allam_model()
        )
        # Extract text answers and log probabilities
        # Append the results

        results.append({
            "question_id": question_id,
            "question": question,
            "original_answer": row.get('Answer', ""),
            "generated_answers": [
                {"text": r['text'], "neg_log_likelihood": r['neg_log_likelihood']}
                for r in generated_results
            ]
        })

   # Step 3: SAVE RESULTS
    results_df = pd.DataFrame(results)
    print(results_df)

    filename = f"generated_answers_Llama3.1-8b{DATASET_CHOICE}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    print(f"\nSaved results to {filename}")
    files.download(filename)

# Run the main function
if __name__ == "__main__":
    main()

Using XOR-TyDiQA
Filtering Arabic QA pairs from XOR-TyDi...
Loaded 708 questions



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.03G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 24.88 MiB is free. Process 2768 has 39.52 GiB memory in use. Of the allocated memory 39.12 GiB is allocated by PyTorch, and 495.50 KiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [6]:
# Main function
def main():
    import pandas as pd
    import json
    from google.colab import files  # If running in Colab

    # Step 1: SELECT AND LOAD DATA
    DATASET_CHOICE = 'xor_tydiqa'  # Choose from above
    FILE_PATH = '/content/xor_dev_full_v1_1.jsonl'  # Only needed for specific datasets

    data = load_qa_dataset(
        DATASET_CHOICE,
        FILE_PATH if DATASET_CHOICE in ['arabicaqa', 'xor_tydiqa', 'AAQAD'] else None
    )

    if not data:
        print("No data loaded. Exiting...")
        return

    # Convert to DataFrame
    df = pd.DataFrame(data)[["question_id", "Question", "Answer"]]
    print(f"Loaded {len(df)} questions\n")

    # Step 2: SET UP MODEL ONCE (not repeatedly)
    print("Setting up ALLaM model...")
    model, tokenizer, device = setup_allam_model()

    # Step 3: GENERATE ANSWERS
    results = []

    for index, row in df.iterrows():
        print(f"Processing question {index+1}/{len(df)}")
        question = row['Question']
        original_answer = row.get('Answer', "")
        question_id = row.get('question_id')

        # Generate 10 answers with log probabilities for each question
        generated_results = generate_answer(
            question=question,
            num_samples=10,
            model=model,
            tokenizer=tokenizer
        )

        # Append the results
        results.append({
            "question_id": question_id,
            "question": question,
            "original_answer": original_answer,
            "generated_answers": [
                {"text": r['text'], "neg_log_likelihood": r['neg_log_likelihood']}
                for r in generated_results
            ]
        })

    # Step 4: SAVE RESULTS
    results_df = pd.DataFrame(results)
    print(results_df)

    filename = f"generated_answers_ALLaM-7B_{DATASET_CHOICE}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    print(f"\nSaved results to {filename}")

    #  download the file
    try:
        files.download(filename)
    except:
        print("File download not available in this environment")

# Run the main function
if __name__ == "__main__":
    main()

Using XOR-TyDiQA
Filtering Arabic QA pairs from XOR-TyDi...
Loaded 708 questions

Setting up ALLaM model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

Processing question 707/708


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processing question 708/708
              question_id                                           question  \
0     5434985093263686694                          ما هي أولى جامعات فنلندا؟   
1     5727312019662110019               ما عدد الدول المطلة على بحر البلطيق؟   
2    -7900394637934786233                                     اين عاش نيوتن؟   
3    -2275140940114918987                            هل زار ابن بطوطة اليمن؟   
4    -7366545647565275586              من هو الرئيس الأول للجمهورية اليمنية؟   
..                    ...                                                ...   
703   5979616361598787767  كم موسم يتألف المسلسل الأمريكي نظرية الانفجار ...   
704   7179923103836317046                         ما هو عدد الفقرات الصدرية؟   
705   8659235659353302418           متى تأسست الوكالة الدولية للطاقة الذرية؟   
706   8086214041898648764                                    من هم الفايكنغ؟   
707  -7492921069579774856                      متى ظهر شرلوك هولمز لأول مرة؟   

           

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>